In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [2]:
# In this code , i have implemented a function calling when gemini api detects when a function call is needed 
from google import genai
from google.genai import types

# Define the function declaration, we are saying the function structue(i.e  function's name, what is does , what parameters it expects, their types,
# descriptions, and which are required for the model to know how to "call" your function.
weather_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco",
            },
        },
        "required": ["location"],
    },
}

# implementing the function 
def get_current_temperature(location):
    return {"location": location, "temperature": "21°C"}

# Client setup
client = genai.Client()

# tools object, which contains one or more function declarations
tools = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[tools])

# Step 1: User asks a question
response = client.models.generate_content(
    model="gemini-2.5-flash",
    # send's user input 
    contents=[types.Content(role="user", parts=[types.Part(text="What's the temperature in Bengaluru?")])],
    config=config,
)

# checking whether the model response contains function call
tool_call = response.candidates[0].content.parts[0].function_call

# Manual Pattern (When we reconstruct history)
# Manually call the function and send response back to the model.
if tool_call and tool_call.name == "get_current_temperature":
    result = get_current_temperature(**tool_call.args)
    print(f"Function execution result: {result}")
    followup_response = client.models.generate_content(
    model="gemini-2.5-flash",
    # we are sending second request( the first query, the function call made by the model, the function's result ) 
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="What's the temperature in Bengaluru?")]
        ),
        types.Content(
            role="model",
            parts=[types.Part(function_call=tool_call)]
        ),
        types.Content(
            role="function",
            parts=[types.Part(
                function_response=types.FunctionResponse(
                    name=tool_call.name,
                    response=result
                )
            )]
        )
    ],
)
    print("Model’s natural language reply:")
    print(followup_response.candidates[0].content.parts[0].text)

else:
    print("No function call found in the response.")


Function execution result: {'location': 'Bengaluru', 'temperature': '21°C'}
Model’s natural language reply:
The current temperature in Bengaluru is 21°C.


In [3]:
# Function calling with thinking
# It tests whether the Gemini model can reason ("think") before answering by showing its internal thought process.
from google import genai
from google.genai import types

client = genai.Client()

grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)

config = types.GenerateContentConfig(
    tools=[grounding_tool],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  # ask model to return reasoning so we know how the model thinks to pick the function/tools
    ),
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List top 10 restaurants in bengaluru which is available in  today(10/23/2025) ",
    config=config,
    
)

#print(response.text)


In [10]:
for part in response.candidates[0].content.parts:
    if getattr(part, "thought", False):
        print("Thoughts:")
        print(part.text)
        print("----")
    else:
        print("Final Answer:")
        print(part.text)


Thoughts:
**Gathering Restaurant Insights**

I'm currently focused on initial restaurant research. My plan involves searching for "best restaurants in Bengaluru" and "top-rated restaurants Bengaluru" to build a solid foundation. Next, I will check restaurant availability for the date October 23, 2025. This should give a practical shortlist. I'm aiming for ten restaurants that consistently get high ratings and are confirmed to be open.



----
Thoughts:
**Narrowing Restaurant Choices**

I've sifted through numerous "best of" lists and articles about new restaurants in Bengaluru, specifically for October 2025, which is quite handy.  My immediate focus is narrowing this down to the top 10, considering their popularity and recent mentions.  While I haven't found direct confirmations of their operating status for today, I'm inferring availability based on the recency of the information.



----
Final Answer:
Here are 10 top-rated restaurants in Bengaluru, with their availability inferred fo

In [4]:
#Inspecting Thought Signatures- mostly it will be displayed when we make function calling, to tell the model what operation performed in before request
# only when both thinking and function calling are enabled and the model generates thoughts.

# checking wheather thought signature is working fine
import base64
# After receiving a response from a model with thinking enabled
# response = client.models.generate_content(...)

# The signature is attached to the response part containing the function call


for idx, part in enumerate(response.candidates[0].content.parts):
    if part.thought_signature:
        print(f"Found thought signature in part {idx}: {part.thought_signature}")

        print(base64.b64encode(part.thought_signature).decode("utf-8"))


In [2]:
# Function calling with thinking
# here, we test how the model reasons and dynamically selects functions to call based on the our request.
from google import genai
from google.genai import types
from pydantic import BaseModel

class cost(BaseModel):
    dish_cost: int
client = genai.Client()


def bhai_biryani(dish:str):
    """ return the cost of the dish, where the dish and it's cost is stored as key value pair in menu dictionary"""
    menu = {'chicken fried rice':100, 'biryani':90, 'half kabab':50}
    return menu.get(dish,0)
def KFC(dish:str):
    """ return the dish cost(key value)"""
    menu={'biryani':200,'fried chicken':600}
    return menu.get(dish)

bhai_biryani_decl = types.FunctionDeclaration.from_callable(callable=bhai_biryani, client=client)
KFC_decl = types.FunctionDeclaration.from_callable(callable=KFC, client=client)

func_tools = types.Tool(function_declarations=[bhai_biryani_decl, KFC_decl])


config = types.GenerateContentConfig(
    tools=[func_tools],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  
    ),
    
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    #contents="what is the total cost if i order two biryani from bhai_biryani  ",
    contents="what is the cost of  biryani in bhai biryani  ",
    config=config,
    
)



for part in response.candidates[0].content.parts:
    if part.function_call:
        func_name = part.function_call.name
        func_args = part.function_call.args
        # Call the corresponding Python function manually
        if func_name == "bhai_biryani":
            result_value = bhai_biryani(**func_args)
        elif func_name == "KFC":
            result_value = KFC(**func_args)
        
        # Wrap in Pydantic model for structured access
        result = cost(dish_cost=result_value)
        print(result.dish_cost)  


90


In [3]:
print("\n--- MODEL THOUGHTS ---")
for part in response.candidates[0].content.parts:
    if getattr(part, "thought", False):
        print(part.text)


--- MODEL THOUGHTS ---
**Thinking Through the Biryani Request**

Okay, so someone's asking about the price of biryani at "bhai biryani."  That's straightforward. I've got this `bhai_biryani` function in the `default_api` that's designed for this exact purpose. It's meant to fetch dish prices from the "bhai biryani" menu. The key is that the function needs the dish name as an argument.  Therefore, to get the price of their desired biryani, I just need to call `bhai_biryani` and pass "biryani" as the `dish` value. Easy peasy.



In [16]:

# parallel function calling
# verify the Gemini model’s parallel function calling capability.


from google import genai
from google.genai import types

def vegeterian(dish:str)->int:
    """ Get the rate of the vegeterian dish

    Args:
        dish : name of the vegeterian dish, e.g: sambar rice

    Returns:
        dictionary where the value is rate
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return {"rate ": menu.get(dish,0)}


def non_vegeterian(dish:str)->int:
    """ Get the rate of the non vegeterian dish

    Args:
        dish : name of the non vegeterian dish, e.g: chicken rice

    Returns:
        dictionary where the value is rate
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return {"rate ": menu.get(dish,0)}

client = genai.Client()

"""
# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(
        mode = 'ANY', allowed_function_names = ["vegeterian","non_vegeterian"]
    )
)
"""
config = types.GenerateContentConfig(

    #passing directly the function itself
    tools = [vegeterian,non_vegeterian],
    #tool_config = tool_config,
    
)


response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "what is the rate of beef rice and curd rice",
    config = config,
)

print(response.text)

The rate of beef rice is 40 and curd rice is 50.


In [17]:
response

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text='what is the rate of beef rice and curd rice'
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          function_call=FunctionCall(
            args={
              <... Max depth ...>: <... Max depth ...>
            },
            name='non_vegeterian'
          ),
          thought_signature=b'\n\xf6\x03\x01\xd1\xed\x8ao\x10\xbb\x10W\xcf\xab\x18\t\xf3d\x93\x1b"%\x1d\xf0\x10[i\x0bz\x9c\x95\x1cJ\xa3\x81\x9a#\xc5\xc5\xe0\x1c\x0c+p\x06\xa8\x18\xe2\xa7Cw lh\t\xcc8\xf0\xbe\xd3\xf5/\xc4\x81\xf96\xe5\xa0qy\xfa\x80t@@\xc2\x9d\xcf\x0b_F\x94\xd0n\\Nw\xd8\xfd\xbeW\xb530\xf4\xd5\x04...'
        ),
        Part(
          function_call=FunctionCall(
            args={
              <... Max depth ...>: <... Max depth ...>
            },
            name='vegeterian'
          )
        ),
      ],
      role='model'
    ),
    Content(
 

In [31]:
# parallel function calling - how can i add structured output(pydantic) in this code:
#here we are testing how Gemini handles parallel function calls and structured outputs together,

from google import genai
from google.genai import types
from pydantic import BaseModel

class dishRate(BaseModel):
    dish:str
    rate:int

def vegeterian(dish:str)->int:
    """ Get the rate of the vegeterian dish

    Args:
        dish : name of the vegeterian dish, e.g: sambar rice

    Returns:
        dishRate object which contains dish name and dish rate
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))


def non_vegeterian(dish:str)->int:
    """ Get the rate of the non vegeterian dish

    Args:
        dish : name of the non vegeterian dish, e.g: chicken rice

    Returns:
        dishRate object which contains dish name and dish rate
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))

client = genai.Client()

"""
# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(
        mode = 'ANY', allowed_function_names = ["vegeterian","non_vegeterian"]
    )
)
"""
# for strutured output the function declaration is optimal(if i provide here, automatic function calling is not take place)
veg_decl = types.FunctionDeclaration.from_callable(callable = vegeterian, client=client)
nonveg_decl = types.FunctionDeclaration.from_callable(callable = non_vegeterian,client=client)
tools = types.Tool(function_declarations=[veg_decl,nonveg_decl])

config = types.GenerateContentConfig(
    
    #tools = [tools],
    tools = [vegeterian,non_vegeterian],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    thinking_config=types.ThinkingConfig(include_thoughts=True),
    response_schema = list[dishRate],
    #response_mime_type = "application/json",
    
    #tool_config = tool_config,
    
)


response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "what is the rate of beef rice and curd rice",
    config = config,
)

"""
print(response)
print(response.text)
structured_result: list[dishRate] = response.parsed
for dish in structured_result:
    print(f"{dish.dish}: {dish.rate}")
"""
print(response.text)
results = []

# iterate over all automatic_function_calling_history items
for history_item in response.automatic_function_calling_history:
    if hasattr(history_item, "parts"):
        for part in history_item.parts:
            if part.function_response:
                func_result = part.function_response.response.get("result")
                if func_result:
                    results.append(func_result)

# results now contains DishRate objects
for dish in results:
    print(f"{dish.dish}: {dish.rate}")



The rate of beef rice is 40 and curd rice is 50.
beef rice: 40
curd rice: 50


In [35]:
# parallel function calling 
# here we test concurrent async workflow with schema-validated results.

from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio
import nest_asyncio
nest_asyncio.apply()

class dishRate(BaseModel):
    dish:str
    rate:int
class allRates(BaseModel):
    dishes:list[dishRate]

def vegeterian(dish:str)->dishRate:
    """ Get the rate of the vegeterian dish
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))


def non_vegeterian(dish:str)->dishRate:
    """ Get the rate of the non vegeterian dish
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))

client = genai.Client()




# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(mode = 'ANY')
)

# for strutured output the function declaration is optimal(if i provide here, automatic function calling is not take place)
#veg_decl = types.FunctionDeclaration.from_callable(callable = vegeterian, client=client)
#nonveg_decl = types.FunctionDeclaration.from_callable(callable = non_vegeterian,client=client)
#tool = types.Tool(function_declarations=[veg_decl,nonveg_decl])
async def main():
    config = types.GenerateContentConfig(
        
        #tools = [tool],
        tools = [vegeterian,non_vegeterian],
        automatic_function_calling=types.AutomaticFunctionCallingConfig( ),
        thinking_config=types.ThinkingConfig(include_thoughts=True),
        #response_schema = allRates,
        #response_mime_type = "application/json",
        response_schema=allRates,
        tool_config = types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(mode="AUTO")
        ),
        
    )

    response = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        #contents="what is the rate of beef rice(non veg) and curd rice(veg)",
        contents ="what is the rate of beef rice(non veg) and curd rice(veg)" ,

        config=config,
    )
    structured_result: allRates = response.parsed
    if response.parsed:
        for dish in response.parsed.dishes:
            print(f"{dish.dish}: {dish.rate}")
    else:
        print("Model did not return a structured response.")
        print(response.text)
    


await main()



Model did not return a structured response.
The rate of beef rice is 40 and curd rice is 50.


In [53]:
# compositional function calling
# This code tests Gemini's compositional function calling — where functions are called sequentially, each depending on prior results. 

from google import genai
from google.genai import types
import json
def pizza_bakery_mani(flavour:str)->dict:
    """ Get the price for a given given flavour"""
    menu = {'chicken':100,'tomato':70,'corn':50,'beef':80}
    return {'flavour':flavour,'price':menu.get(flavour)}

def pizza_bakery_mani_gst(price:int)->dict:
    """ return the gst for this price"""
    return {"gst_price":price*0.10}

client = genai.Client()
config = types.GenerateContentConfig(
    tools=[pizza_bakery_mani, pizza_bakery_mani_gst ]
)


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Get me the price and gst for chicken(flavour) pizza from pizza bakery mani",
    config=config,
)

print(response.text)


The price for chicken pizza is 100 and the GST for it is 10.


In [6]:
# parallel function calling 

from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio


class dishRate(BaseModel):
    dish:str
    rate:int
class allRates(BaseModel):
    dishes:list[dishRate]

def vegeterian(dish:str)->int:
    """ Get the rate of the vegeterian dish
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))


def non_vegeterian(dish:str)->int:
    """ Get the rate of the non vegeterian dish
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))

client = genai.Client()




# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(mode = 'ANY')
)

# for strutured output the function declaration is optimal(if i provide here, automatic function calling is not take place)
#veg_decl = types.FunctionDeclaration.from_callable(callable = vegeterian, client=client)
#nonveg_decl = types.FunctionDeclaration.from_callable(callable = non_vegeterian,client=client)
#tool = types.Tool(function_declarations=[veg_decl,nonveg_decl])

config = types.GenerateContentConfig(
    
    #tools = [tool],
    tools = [vegeterian,non_vegeterian],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    thinking_config=types.ThinkingConfig(include_thoughts=True),
    response_schema = allRates,
    #response_mime_type = "application/json",
    
    tool_config = types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
    
)

"""
response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "what is the rate of beef rice(non veg) and curd rice(veg)",
    config = config,
)
"""
async def main():
    response = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        contents="what is the rate of beef rice(non veg) and curd rice(veg)",
        config=config,
    )
    results = []
    if response.automatic_function_calling_history:
        # Consider only the last history item (most recent function call response)
        latest_history = response.automatic_function_calling_history[-1]
        if hasattr(latest_history, "parts"):
            for part in latest_history.parts:
                if part.function_response:
                    func_result = part.function_response.response.get("result")
                    if func_result:
                        results.append(func_result)
    
    # Now print results without duplication
    seen = set()
    for dish in results:
        key = (dish.dish, dish.rate)
        if key not in seen:
            print(f"{dish.dish}: {dish.rate}")
            seen.add(key)
    """
    print("Automatic Function Calls:\n", response.automatic_function_calling_history)
    
    if response.parsed:
        structured_result: allRates = response.parsed
        for dish in structured_result.dishes:
            print(f"{dish.dish}: {dish.rate}")
    else:
        print("Structured parse failed, using raw extraction.")
        results = []
        for history_item in response.automatic_function_calling_history:
            if hasattr(history_item, "parts"):
                for part in history_item.parts:
                    if part.function_response:
                        func_result = part.function_response.response.get("result")
                        if func_result:
                            results.append(func_result)
        for item in results:
            print(f"{item.dish}: {item.rate}")
    """


#asyncio.run(main())
await main()
"""
print(response.text)
results = []

# iterate over all automatic_function_calling_history items
for history_item in response.automatic_function_calling_history:
    if hasattr(history_item, "parts"):
        for part in history_item.parts:
            if part.function_response:
                func_result = part.function_response.response.get("result")
                if func_result:
                    results.append(func_result)

# results now contains DishRate objects
for dish in results:
    print(f"{dish.dish}: {dish.rate}")

"""

beef rice: 40
curd rice: 50


'\nprint(response.text)\nresults = []\n\n# iterate over all automatic_function_calling_history items\nfor history_item in response.automatic_function_calling_history:\n    if hasattr(history_item, "parts"):\n        for part in history_item.parts:\n            if part.function_response:\n                func_result = part.function_response.response.get("result")\n                if func_result:\n                    results.append(func_result)\n\n# results now contains DishRate objects\nfor dish in results:\n    print(f"{dish.dish}: {dish.rate}")\n\n'

In [19]:
# here we testing structured compositional function-calling test

import os
from google import genai
from google.genai import types
from pydantic import BaseModel

# -----------------------------
# Define Pydantic schema
# -----------------------------
class ThermostatResult(BaseModel):
    location: str
    temperature: int
    #thermostat_setting: int
    status: str

#--------------------------------
# Define functions
# -----------------------------
def get_weather_forecast(location: str) -> dict:
    """Gets the current weather temperature for a given location."""

    print(f"Tool Call: get_weather_forecast(location={location})")
    print("Tool Response: {'temperature': 25, 'unit': 'celsius'}")
    return {"temperature": 25, "unit": "celsius"}  

def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    print("Tool Response: {'status': 'success'}")
    return {"status": "success"}

# -----------------------------
# 3️⃣ Configure client + schema
# -----------------------------
client = genai.Client()

config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    response_schema=ThermostatResult,   
    thinking_config=types.ThinkingConfig(include_thoughts=False),
    
)

# -----------------------------
# 4️⃣ Run model
# -----------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    #contents="If it's warmer than 20°C in London, set the thermostat to 20°C, otherwise set it to 18°C.",
    #contents = """
    #        If it's warmer than 20°C in London, set thermostat to 20°C, otherwise 18°C.
    #        Respond only in valid JSON matching  schema
    #        """,
    contents = " Get weather forecast for chennai and  set thermostat temperature from getting value from weather forecast for chennai",
    config=config,
)

# -----------------------------
# 5️⃣ Access structured output
# -----------------------------
if response.parsed:
    result: ThermostatResult = response.parsed
    print("✅ Structured Result:")
    print(result.json(indent=2))

    # You can reuse later like:
    print("\nLater use example:")
    print(f"Thermostat was set to {result.thermostat_setting}°C in {result.location}")
else:
    print("Model did not return structured output.")
    print(response.text)


Tool Call: get_weather_forecast(location=chennai)
Tool Response: {'temperature': 25, 'unit': 'celsius'}


Tool Call: set_thermostat_temperature(temperature=25)
Tool Response: {'status': 'success'}


Model did not return structured output.
The weather forecast for Chennai is 25 degrees Celsius. I have set the thermostat temperature to 25 degrees Celsius.


In [25]:
collected_data = {
    "location": "Chennai",
    "temperature": None,
    "status": None,
}

for content in response.automatic_function_calling_history:
    for part in content.parts:
        if hasattr(part, "function_response") and part.function_response:
            func_name = part.function_response.name
            func_resp = part.function_response.response or {}
            print(func_name, func_resp)
            if func_name == "get_weather_forecast":
                collected_data["temperature"] = func_resp['result'].get("temperature")
            elif func_name == "set_thermostat_temperature":
                collected_data["status"] = func_resp['result'].get("status")



get_weather_forecast {'result': {'temperature': 25, 'unit': 'celsius'}}
set_thermostat_temperature {'result': {'status': 'success'}}


In [26]:
collected_data

{'location': 'Chennai', 'temperature': 25, 'status': 'success'}

In [33]:
json_str = result.model_dump_json(indent=2)
print(json_str)


{
  "location": "Chennai",
  "temperature": 25,
  "status": "success"
}


In [34]:
dict_data = result.model_dump()
print(dict_data)


{'location': 'Chennai', 'temperature': 25, 'status': 'success'}


In [35]:
print(f"Location: {result.location}")
print(f"Temperature: {result.temperature}")
print(f"Status: {result.status}")


Location: Chennai
Temperature: 25
Status: success


In [48]:
# this is code is same as previous of compositional with structured schema which includes this new code contains, validation error 
#with our schema and model reasoning for choosing the function
from google import genai
from google.genai import types
from pydantic import BaseModel, ValidationError


class ThermostatResult(BaseModel):
    location: str
    current_temperature: int
    status: str



def get_weather_forecast(location: str) -> dict:
    """Simulate weather data retrieval"""
    print(f"[Tool Call] get_weather_forecast({location})")
    return {"temperature": 25, "unit": "celsius"} 


def set_thermostat_temperature(temperature: int) -> dict:
    """Simulate thermostat configuration"""
    print(f"[Tool Call] set_thermostat_temperature({temperature})")
    return {"status": "success"}



client = genai.Client()

config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    thinking_config=types.ThinkingConfig(include_thoughts=True),  
    response_schema=ThermostatResult,  
)

contents = (
    "If it's warmer than 20°C in London, set the thermostat to 20°C. "
    "Otherwise, set it to 18°C. Return the output in the required structure."
)


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)


if response.parsed:
    #  Automatic structured parsing success
    result: ThermostatResult = response.parsed
    print("\n Structured Output from Gemini:")
    print(result.json(indent=2))

else:
    # Automatic parse failed — perform manual recovery
    print("\n Structured parse failed. Attempting manual parsing...\n")

    # Extract relevant function call responses
    results = []
    for candidate in response.candidates:
        for part in candidate.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                print("name ", part.function_response.name)
                response_data = part.function_response.response.get("result")
                if response_data:
                    try:
                        validated = ThermostatResult(**response_data)
                        results.append(validated)
                    except ValidationError as e:
                        print(f"Schema Validation Error:\n{e}")

    if results:
        print(" Recovered Structured Output:")
        for r in results:
            print(r.json(indent=2))
    else:
        print(" No valid structured data found. Model text output:\n")
        print(response.text)





[Tool Call] get_weather_forecast(London)


[Tool Call] set_thermostat_temperature(20)



 Structured parse failed. Attempting manual parsing...

 No valid structured data found. Model text output:

The thermostat has been set to 20°C.


In [49]:
response

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text="If it's warmer than 20°C in London, set the thermostat to 20°C. Otherwise, set it to 18°C. Return the output in the required structure."
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          text="""**Controlling the Thermostat Based on London's Weather**

Okay, so the goal is to intelligently manage this thermostat in response to the weather in London.  My thinking is, first things first: I *need* the weather forecast for London.  I'll call the `get_weather_forecast` function, specifying "London" as the location.  That'll give me the data I need.

Next, I'll extract the temperature reading from that weather data. Now comes the decision-making part.  I'm going to compare that temperature to 20°C. If it's warmer than 20°C, I want to set the thermostat to 20°C; if it's colder or exactly 20°C, then 18°C. Simple.

Finally, th

In [22]:
from google import genai
from google.genai import types

client = genai.Client()

# -------------------------------
# Step 1: Define the function
# -------------------------------
restaurant_rating_function = {
    "name": "get_restaurant_rating",
    "description": "Fetches the average rating for a given restaurant in Bengaluru.",
    "parameters": {
        "type": "object",
        "properties": {
            "restaurant_name": {
                "type": "string",
                "description": "The name of the restaurant.",
            },
        },
        "required": ["restaurant_name"],
    },
}

def get_restaurant_rating(restaurant_name: str):
    # Mock data – in real case, this could query Zomato/Google Maps API
    mock_ratings = {
        "Farmlore": 4.8,
        "Jamavar": 4.6,
        "Karavalli": 4.7,
        "ZLB23": 4.5,
        "Copitas": 4.6,
        "The Fatty Bao": 4.4,
        "Rim Naam": 4.7,
        "Mavalli Tiffin Room": 4.3,
        "Yuki Cocktail Bar & Kitchen": 4.5,
        "Lotus Pavilion": 4.4,
    }
    return {"restaurant": restaurant_name, "rating": mock_ratings.get(restaurant_name, "N/A")}

# -------------------------------
# Step 2: Configure with thinking
# -------------------------------
tools = types.Tool(function_declarations=[restaurant_rating_function])

config = types.GenerateContentConfig(
    tools=[tools],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  # Ask the model to return its reasoning
    ),
)

# -------------------------------
# Step 3: Ask the model
# -------------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the rating of Farmlore restaurant in Bengaluru?",
    config=config,
)

# -------------------------------
# Step 4: Inspect response
# -------------------------------
for idx, part in enumerate(response.candidates[0].content.parts):
    if part.thought_signature:
        print(f"Found thought signature in part {idx}: {part.thought_signature}")

    if part.function_call:
        print(f"\nModel requested function: {part.function_call.name}")
        print(f"Arguments: {part.function_call.args}")

        # Execute the function
        result = get_restaurant_rating(**part.function_call.args)
        print(f"Function execution result: {result}")

        # -------------------------------
        # Step 5: Send function result back
        # -------------------------------
        followup_response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                response.candidates[0].content,  # include model’s function call turn
                types.Content(
                    role="function",
                    parts=[types.Part(
                        function_response=types.FunctionResponse(
                            name=part.function_call.name,
                            response=result
                        )
                    )]
                )
            ],
        )

        print("\nFinal model reply:")
        print(followup_response.text)


Found thought signature in part 1: b'\n\xa7\x02\x01\xd1\xed\x8ao\x83\x12\x97\xea+\xfc5,\xe1:\xbd\xd2T-\x04Ux\'\x8cLbs\x96\xff\xcaT\x03\\\x9f\xd7\x93=\nTM\x93\x01OgNS\\\x0e\xa7\xd1\x02c\xea\x1e\xd9\x06\xac\xb0~\x93\xae\xe2^\x85L=\xb0\xcd\x9a\x08x\x91\xd7\xa4F\xa6e\xf2\xe2\xc4\x02\xa4\xf4\xf0\x13P\x8e\xbd\x88TL\xdef\xa9\x8aW&\xbb\x15\xad\x87g\xb1\x10\x87\xacM\xe1\xaed2T\x84\x98.\x16\xa5\xc8\xd5\x0bQ\x91\xa0;\xb3\xa0\x06\xdc\x08\xec\xb7\x98\xde\xf9\xd2\x00#\xb7\xe6\x82\xfb"\xbej\x9dyFY\x97\x94\xddg_\xa6V.f\xf4\xf0p\xce\x8dt\x92!\xb159D\x16\xfb\xac\xa8\xc9\x05\xf8\xb3\xf3~\x11!D-\xad\x04\xf1rY\xa4H\xe2\x804\x1e\x01\xba:RU\xe4\xe7\x9d\xc7G\xd8-(\x0e`\xb6\x9fM\x18\xbd\x86\xe1py.-\xe5\xbc\x8a\xcd;\xdc7\xeb^\r\xbf\x99\x05K\x8b]\x8b\xbb4\xe9\xa8\x98<*8l\x9e\xbfd{\xead\x14\x08\xd3\xa3\x1b\x83v\x92H\xaa\xc9Ox\x1a\xb2_\xd6.\x03\x89\xd0>\t\x08E\xd2C\xa4\xaa\xfeQ\x97\xff\x1e\x8c\x9a\xf5\x97p\xfa,L3'

Model requested function: get_restaurant_rating
Arguments: {'restaurant_name': 'Farmlore'}
Function 

In [6]:
# Automatic function schema declaration

from google import genai
from google.genai import types

def multiply(a: float, b: float):
    """Returns a * b."""
    return a * b

client = genai.Client()

#  FunctionDeclarations from Python functions directly using types.FunctionDeclaration.from_callable(client=client, callable=your_function).
fn_decl = types.FunctionDeclaration.from_callable(callable=multiply, client=client)

print(fn_decl)
# to_json_dict() provides a clean JSON representation.
print(fn_decl.to_json_dict())

behavior=None description='Returns a * b.' name='multiply' parameters=Schema(
  properties={
    'a': Schema(
      type=<Type.NUMBER: 'NUMBER'>
    ),
    'b': Schema(
      type=<Type.NUMBER: 'NUMBER'>
    )
  },
  required=[
    'a',
    'b',
  ],
  type=<Type.OBJECT: 'OBJECT'>
) parameters_json_schema=None response=None response_json_schema=None
{'description': 'Returns a * b.', 'name': 'multiply', 'parameters': {'properties': {'a': {'type': 'NUMBER'}, 'b': {'type': 'NUMBER'}}, 'required': ['a', 'b'], 'type': 'OBJECT'}}
